In [19]:
import pyspark
import findspark
findspark.init()
import os
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
findspark.find()

'C:\\spark\\spark-3.2.4-bin-hadoop2.7'

In [33]:
# spark session
sc = SparkContext('local', 'calif-coll-data-union')
spark = SparkSession(sc)

In [34]:
path = 'C:\\Users\\RoiMinuit\\Desktop\\data\\switrs_raw_csvs'

# find folders
folders = []
for folder in os.listdir(path):
    folder = os.path.join(path, folder)
    folders.append(folder)
    
# collect file name from each folder
f_names = []
for folder in folders:
    for f in os.listdir(folder):
        if f.endswith('CollisionRecords.txt'):
            f_names.append(os.path.join(folder, f))

In [35]:
# check shape of each df to ensure compatibility and store df
all_data = []
for df in f_names:
    df = spark.read.csv(df, header=True)
    print(df.count(), len(df.columns))
    all_data.append(df)

7314602 76
7437041 76
8293457 76
5494226 76
5293361 76


In [36]:
# combine dfs
if all_data:
    one_data = all_data[0]
    for df in all_data[1:]:
        one_data = one_data.union(df)

In [41]:
print(one_data.count(), len(one_data.columns))
output_path = "all_collission_data.txt"
one_data.write.mode('overwrite').text(output_path)

33832687 76


AnalysisException: Text data source supports only a single column, and you have 76 columns.

In [32]:
# close session when needed
sc.stop()